In [1]:
%run nb_config

StatementMeta(, 971d68ec-0984-4a9d-a091-4886b2abd2ee, 3, Finished, Available, Finished)

nb_config Loaded


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from datetime import datetime, timezone
import uuid
import traceback

StatementMeta(, 971d68ec-0984-4a9d-a091-4886b2abd2ee, 4, Finished, Available, Finished)

In [3]:
spark = SparkSession.builder.getOrCreate()

StatementMeta(, 971d68ec-0984-4a9d-a091-4886b2abd2ee, 5, Finished, Available, Finished)

In [4]:
# Log kayıtlarının tutulacağı yer
LOG_PATH = f"{BASE_PATH}/logs"
RUN_ID = str(uuid.uuid4())

StatementMeta(, 971d68ec-0984-4a9d-a091-4886b2abd2ee, 6, Finished, Available, Finished)

In [5]:
def get_timestamp():
    return datetime.now(timezone.utc)

# Log Şeması
log_schema = StructType([
    StructField("timestamp", TimestampType(), True),
    StructField("level", StringType(), True),
    StructField("source", StringType(), True),
    StructField("process", StringType(), True),
    StructField("message", StringType(), True),
    StructField("details", StringType(), True),
    StructField("run_id", StringType(), True)
])

def write_log(level, source, process, message, details=None):
    """Log verisini Data Lake'e yazar."""
    try:
        log_data = [{
            "timestamp": get_timestamp(),
            "level": level,
            "source": source,
            "process": process,
            "message": message,
            "details": details,
            "run_id": RUN_ID
        }]
        
        df = spark.createDataFrame(log_data, schema=log_schema)
        
        # Delta veya Parquet olarak append modunda yaz
        # Hata almamak için Delta formatını tercih ediyoruz, yoksa parquet kullanın.
        # nb_config içinde FILE_FORMAT yoksa varsayılan parquet'tir.
        df.write.mode("append").parquet(LOG_PATH)
        
        # Konsola da yazdıralım ki anlık görebilesiniz
        print(f"[{level}] {source} - {message}")
        
    except Exception as e:
        print(f"Log yazma hatası: {e}")

# --- EKSİK OLAN KISIM: Logger Sınıfı ve get_logger Fonksiyonu ---

class LakehouseLogger:
    def __init__(self, name):
        self.source = name

    def info(self, message):
        # Process ismi varsayılan olarak 'ETL' atanır, detaylandırılabilir.
        write_log("INFO", self.source, "General", message)

    def warning(self, message):
        write_log("WARNING", self.source, "General", message)

    def error(self, message, error_details=None):
        # Eğer string bir detay verildiyse onu kullan, yoksa None
        details = str(error_details) if error_details else None
        write_log("ERROR", self.source, "General", message, details)

    def exception(self, message):
        # Python'daki son hatayı (traceback) yakalar
        details = traceback.format_exc()
        write_log("ERROR", self.source, "Exception", message, details)

def get_logger(name):
    """Notebooklarda logger nesnesi oluşturmak için çağrılan fonksiyon."""
    return LakehouseLogger(name)

print("nb_logging Loaded")

StatementMeta(, 971d68ec-0984-4a9d-a091-4886b2abd2ee, 7, Finished, Available, Finished)

nb_logging Loaded
